In [6]:
import sys
import pandas as pd
import torch 
import os 

sys.path.append("../..")
# For retrieval

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from sentence_transformers import CrossEncoder
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chunkers import TableExtractor

from financerag.tasks import MultiHiertt

## Read Data

In [ ]:
task = MultiHiertt()

In [8]:
queries = task.queries
query_df = pd.DataFrame(queries.values(), index=queries.keys(), columns=["query"])

In [9]:
documents = task.corpus
documents_df = pd.DataFrame(documents.values(), index=documents.keys(), columns=["title", "text"])

In [353]:
def extract(text):
    array = text.split("\n")
    paragraph_lines = []
    table_lines = []

    for i, line in enumerate(array):

        if "|" not in line:
            paragraph_lines.append(i)
        else:
            table_lines.append(i)
        
    paragraphs = []

    paragraph = []

    for i in range(len(paragraph_lines)):

        if i == (len(paragraph_lines) - 1):
            paragraph.append(paragraph_lines[i])
            paragraphs.append(paragraph)
            break

        if (paragraph_lines[i] + 1) == paragraph_lines[i+1]:
            paragraph.append(paragraph_lines[i])

        else:
            paragraph.append(paragraph_lines[i])
            paragraphs.append(paragraph)
            paragraph = []

    paragraphs
    paragraphs_ = []

    for paragraph in paragraphs:
        start = paragraph[0]
        end = paragraph[-1]

        paragraphs_.append("\n".join(text.split("\n")[start:end+1]))

    tables = []

    table = []

    for i in range(len(table_lines)):
        
        if i == (len(table_lines) - 1):
            table.append(table_lines[i])
            tables.append(table)
            break

        if (table_lines[i] + 1) == table_lines[i+1]:
            table.append(table_lines[i])

        else:
            table.append(table_lines[i])
            tables.append(table)
            table = []

    tables
    tables_ = []

    for table in tables:
        start = table[0]
        end = table[-1]

        tables_.append("\n".join(text.split("\n")[start:end+1]))
        
    return paragraphs_, tables_

In [354]:
def purge(row):

    return_list = []

    for cell in row:
        if cell.strip() == "":
            continue

        if any(keyword in cell.strip().lower().split(" ")for keyword in ["(in", "millions", "thousands", "in", "dollars", "note", "thousands)", "millions)", "dollars)", "year", "ended", "december"]):
            continue    
            
        else:
            return_list.append(cell)
    
    return return_list

In [502]:
def clean(row):

    return_list = []

    for cell in row:

        if any(keyword in cell.strip().lower().split(" ")for keyword in ["(in", "millions", "thousands", "in", "dollars", "note", "thousands)", "millions)", "dollars)", "year", "ended", "december",
                                                                         "january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november",
                                                                         "(millions), (thousands)", "(dollars)", "millions), (dollars)"]):
            continue    
            
        else:
            return_list.append(cell)
    
    return return_list

In [503]:
def parse_table(table):
    
    levels = []

    for row in table:
        row = row.split(" ")
        level = 0
        for i, cell in enumerate(row):
            if cell == "|":
               level += 1
            elif cell == "":
                continue
            else:
                break
        levels.append(level)

    headers = []

    for level, row in zip(levels, table):
        if level > 1:
            table.remove(row)
            row = row.split("|")
            row = purge(row)
            headers.append(row)
    
    if headers == []:
        columns = [ column.strip() for column in table[0].split("|") if column.strip() != ""]
        columns = clean(columns)
        table.remove(table[0])
    else:
        columns_set = False
        
        for header in headers:
            if header != []:
                columns = header
                columns_set = True
                
        if not columns_set:
            columns = [ column.strip() for column in table[0].split("|") if column.strip() != ""]
            columns = clean(columns)
            table.remove(table[0])

    rows = []

    for row in table:
        row = row.split("|")[1:-1]
        row = clean(row)
        rows.append(row)
    
    if len(columns) < len(rows[0]):
        columns = [i for i in range(len(rows[0]) - len(columns))] + columns

    return pd.DataFrame(rows, columns=columns)

In [504]:
document_tables = {}

for id, text in documents_df["text"].items():
    try:
        paragraphs, tables = extract(text)
        
        if len(tables) == 0:
            document_tables[id] = []

        table_dict = {}

        for i, table in enumerate(tables):
            table = table.split("\n")
            table = parse_table(table)
            table_dict[f"Table-{i}"] = table
            document_tables[id] = table_dict

    except Exception as e:
        print(id)
        print(e)
        pass

d8d24b238
1 columns passed, passed data had 4 columns
d8cc5eb18
3 columns passed, passed data had 4 columns
d8aa1dd4c
4 columns passed, passed data had 6 columns
d862755bc
6 columns passed, passed data had 13 columns
d8d64e6fa
4 columns passed, passed data had 5 columns
d8c6d6b6e
4 columns passed, passed data had 5 columns
d89532536
8 columns passed, passed data had 9 columns
d8eca4d96
8 columns passed, passed data had 9 columns
d8c95a548
5 columns passed, passed data had 6 columns
d87b35a48
10 columns passed, passed data had 11 columns
d8e0b4d24
5 columns passed, passed data had 6 columns
d8ea69f72
3 columns passed, passed data had 5 columns
d87eff462
3 columns passed, passed data had 4 columns
d89b1abec
2 columns passed, passed data had 3 columns
d8c0c05ae
1 columns passed, passed data had 3 columns
d8e0599ba
2 columns passed, passed data had 3 columns
d8945848a
3 columns passed, passed data had 4 columns
d8ef60922
1 columns passed, passed data had 5 columns
d87851e1c
1 columns passe

In [505]:
len( list(document_tables.keys()))

8745

In [506]:
documents_df.shape

(10475, 2)

In [511]:
import random

# Filter out documents that have a 'Table-1'
table_1_docs = {k: v for k, v in document_tables.items() if 'Table-1' in v}

# Select a random document with 'Table-1'
random_doc_id = random.choice(list(table_1_docs.keys()))
random_table_1 = table_1_docs[random_doc_id]['Table-1']

text = documents_df.loc[random_doc_id, 'text']
print(text)

print(random_doc_id)
random_table_1

The following table presents the results of operations of our Annuities segment:
|  | Years Ended December 31, |  |  |
|  | 2009 | 2008 | Change |
|  | (in millions, except percentages) |
| Revenues |  |  |  |  |
| Management and financial advice fees | $438 | $478 | $-40 | -8% |
| Distribution fees | 247 | 275 | -28 | -10 |
| Net investment income | 1,323 | 652 | 671 | NM |
| Premiums | 104 | 85 | 19 | 22 |
| Other revenues | 153 | 128 | 25 | 20 |
| Total revenues | 2,265 | 1,618 | 647 | 40 |
| Banking and deposit interest expense | — | — | — | — |
| Total net revenues | 2,265 | 1,618 | 647 | 40 |
| Expenses |  |  |  |  |
| Distribution expenses | 211 | 207 | 4 | 2 |
| Interest credited to fixed accounts | 759 | 646 | 113 | 17 |
| Benefits, claims, losses and settlement expenses | 418 | 269 | 149 | 55 |
| Amortization of deferred acquisition costs | 37 | 576 | -539 | -94 |
| General and administrative expense | 192 | 207 | -15 | -7 |
| Total expenses | 1,617 | 1,905 | -288 | -15 |
| P

,Buildings,"42,695","21,821"
0,,2006,2005
1,Computer and other equipment,"$376,999","$351,253"
2,Furniture and fixtures,"27,535","29,936"
3,Land,"20,414","42,754"
4,Leasehold improvements,"55,321","71,217"
5,,"522,964","516,981"
6,Less accumulated depreciation and amortization,"-382,304","-346,786"
7,"Total property and equipment, net-1","$140,660","$170,195"


In [512]:
text = documents_df.loc[random_doc_id, 'text']
print(text)

The following table presents the results of operations of our Annuities segment:
|  | Years Ended December 31, |  |  |
|  | 2009 | 2008 | Change |
|  | (in millions, except percentages) |
| Revenues |  |  |  |  |
| Management and financial advice fees | $438 | $478 | $-40 | -8% |
| Distribution fees | 247 | 275 | -28 | -10 |
| Net investment income | 1,323 | 652 | 671 | NM |
| Premiums | 104 | 85 | 19 | 22 |
| Other revenues | 153 | 128 | 25 | 20 |
| Total revenues | 2,265 | 1,618 | 647 | 40 |
| Banking and deposit interest expense | — | — | — | — |
| Total net revenues | 2,265 | 1,618 | 647 | 40 |
| Expenses |  |  |  |  |
| Distribution expenses | 211 | 207 | 4 | 2 |
| Interest credited to fixed accounts | 759 | 646 | 113 | 17 |
| Benefits, claims, losses and settlement expenses | 418 | 269 | 149 | 55 |
| Amortization of deferred acquisition costs | 37 | 576 | -539 | -94 |
| General and administrative expense | 192 | 207 | -15 | -7 |
| Total expenses | 1,617 | 1,905 | -288 | -15 |
| P

In [513]:
paragraphs, tables = extract(text)
for table in tables:
   table = parse_table(table.split("\n"))
   break
table

,Management and financial advice fees,$438,$478,$-40,-8%
0,,2009,2008,Change,None
1,Revenues,,,,
2,Distribution fees,247,275,-28,-10
3,Net investment income,"1,323",652,671,NM
4,Premiums,104,85,19,22
5,Other revenues,153,128,25,20
6,Total revenues,"2,265","1,618",647,40
7,Banking and deposit interest expense,—,—,—,—
8,Total net revenues,"2,265","1,618",647,40
9,Expenses,,,,
